In [ ]:
import os
import json
import dotenv
from crewai import Crew, Agent, Task, tools, Process
from crewai.tools import tool
from crewai_tools.tools.rag.rag_tool import RagTool
from crewai_tools import FileWriterTool
from typing import List
from langchain_community.chat_models import ChatLiteLLM
import litellm
# import XDATA
from XDATA import create_x_data_collector_task

dotenv.load_dotenv()

In [ ]:
@tool("X Tweet Data Collector")
def collect_x_tweets(creators: List[str]) -> dict:
    """
    Collect comprehensive tweet data for specified X (Twitter) creators.

    Args:
        creators (List[str]): List of X creator usernames to collect tweets from
    
    Returns:
        dict: Structured tweet data with metadata and collected tweets
    """
    return create_x_data_collector_task(creators)

class XSentimentAnalysisProject:
    def __init__(self):
        """
        Initialize the X Sentiment Analysis Project with configurable LLM.
        
        Args:
            model_provider (str): LLM provider (openai, anthropic, etc.)
        """
        # Configure LLM with litellm for flexibility
        self.llm = ChatLiteLLM(model="gpt-3.5-turbo",api_key=os.environ.get("OPENAI_API_KEY"))

    def create_x_data_collector_agent(self):
        """
        Create an agent responsible for collecting X (Twitter) data.
    
        Returns:
            Agent: X Data Collector Agent
        """
        return Agent(
            role='X Data Collector',
            goal='Retrieve comprehensive tweet data from target X creators',
            backstory="""You are a sophisticated data gathering expert specialized 
            in social media content extraction. Your mission is to collect tweets, 
            analyze their context, and prepare structured data for further analysis.""",
            tools=[
                collect_x_tweets
            ],
            verbose=True,
            allow_delegation=False
        )
    
    def create_sentiment_analyzer_agent(self):
        """
        Create an agent for sentiment and contextual analysis of collected tweets.
        
        Returns:
            Agent: Sentiment Analyzer Agent
        """
        return Agent(
            role='Social Media Sentiment Analyst',
            goal='Perform deep sentiment and thematic analysis on collected tweet data',
            backstory="""You are an expert in natural language processing and 
            social media content interpretation. Your advanced analytical skills 
            help extract nuanced insights from digital communication.
            You have access to a rag tool to read information and a text file writing tool to write the results into a text file""",
            tools=[
                RagTool(),  # CrewAI's RAG tool for contextual analysis
                FileWriterTool()
            ],
            verbose=True,
            allow_delegation=False
        )

    def create_data_collection_task(self, data_collector_agent, creators):
        """
        Design a task for collecting tweet data from specified creators.
    
        Args:
            data_collector_agent (Agent): X Data Collector Agent
            creators (list): List of X creator usernames
    
        Returns:
            Task: Data collection task
        """
        return Task(
            description=f"""Collect tweets from the following X creators: {', '.join(creators)}
            - Use XDataScraper in the create_x_data_collector_taskto scrape tweets
            - Collect comprehensive tweet metadata including:
              * Username
              * Tweet content

            - Ensure data is stored in a structured JSON format
            - Respect data privacy and platform terms of service""",
            agent=data_collector_agent,
            output_file='./tweets.json',
            expected_output="""Comprehensive JSON structure containing:
            {
                "data_source": "X (Twitter)",
                "users_analyzed": ["username1", "username2"],
                "total_tweets_collected": 100,
                "tweets": [
                    {
                        "username": "creator_username",
                        "content": "Tweet text content"
                    }
                ]
            }"""
        )

    def create_sentiment_analysis_task(self, sentiment_analyzer_agent, collected_data):
        """
        Design a task for sentiment and thematic analysis.
        
        Args:
            sentiment_analyzer_agent (Agent): Sentiment Analyzer Agent
            collected_data (dict): JSON data of collected tweets
        
        Returns:
            Task: Sentiment analysis task
        """
        return Task(
            description="""Perform comprehensive sentiment and thematic analysis on collected tweets
            - Analyze sentiment for each tweet (positive, negative, neutral)
            - Identify dominant themes and topics across tweets
            - Generate a detailed report with insights and save to a text file
            - Include visualizations of sentiment distribution
            - Provide actionable insights about each creator's content""",
            agent=sentiment_analyzer_agent,
            context=[collected_data],
            expected_output="""text file Report with:
            1. Overall sentiment analysis
            2. Creator-specific insights
            3. Thematic breakdown
            4. Engagement metrics visualization"""
        )

    def run_analysis(self, creators):
        """
        Execute the complete X sentiment analysis workflow.
        
        Args:
            creators (list): List of X creator usernames
        """
        

        # Initialize agents
        data_collector = self.create_x_data_collector_agent()
        sentiment_analyzer = self.create_sentiment_analyzer_agent()

        collected_data = self.create_data_collection_task(
            data_collector,
            creators
        )

        # Create sentiment analysis task with the collected data
        sentiment_analysis_task = self.create_sentiment_analysis_task(
            sentiment_analyzer, 
            collected_data  # Pass the actual collected data instead of an empty placeholder
        )

        # Create crew and execute workflow
        crew = Crew(
            agents=[data_collector,sentiment_analyzer],  # Remove data collector since scraping is now pre-done
            tasks=[collected_data,sentiment_analysis_task],
            process=Process.sequential,
            verbose=True
        )

        # Run the workflow
        result = crew.kickoff()
        return result

def main():
    # Example X creators to analyze
    x_creators = [
        'elonmusk', 'naval', 'paulg', 
        'sama', 'lex_fridman', 'balajis'
    ]

    # Initialize and run project
    project = XSentimentAnalysisProject()
    analysis_results = project.run_analysis(x_creators)
    
    # Optional: Save or further process results
    print("Analysis Complete:", analysis_results)



In [ ]:
if __name__ == "__main__":
    main()